In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [3]:
df = pd.read_csv("D:\Semester-7\Data Visualisation\energy-dashboard\data\owid_energy.csv")

for col in df.columns:
    print(col)

country
year
iso_code
population
gdp
biofuel_cons_change_pct
biofuel_cons_change_twh
biofuel_cons_per_capita
biofuel_consumption
biofuel_elec_per_capita
biofuel_electricity
biofuel_share_elec
biofuel_share_energy
carbon_intensity_elec
coal_cons_change_pct
coal_cons_change_twh
coal_cons_per_capita
coal_consumption
coal_elec_per_capita
coal_electricity
coal_prod_change_pct
coal_prod_change_twh
coal_prod_per_capita
coal_production
coal_share_elec
coal_share_energy
electricity_demand
electricity_demand_per_capita
electricity_generation
electricity_share_energy
energy_cons_change_pct
energy_cons_change_twh
energy_per_capita
energy_per_gdp
fossil_cons_change_pct
fossil_cons_change_twh
fossil_elec_per_capita
fossil_electricity
fossil_energy_per_capita
fossil_fuel_consumption
fossil_share_elec
fossil_share_energy
gas_cons_change_pct
gas_cons_change_twh
gas_consumption
gas_elec_per_capita
gas_electricity
gas_energy_per_capita
gas_prod_change_pct
gas_prod_change_twh
gas_prod_per_capita
gas_produ

In [4]:
cols = [
    "country",
    "year",
    "primary_energy_consumption",
    "renewables_share_energy",
    "fossil_share_energy",
    "solar_share_energy",
    "wind_share_energy",
    "hydro_share_energy"
]

df = df[cols]

df = df[df["year"] >= 1990]

exclude = [
    "World", "Asia", "Europe", "Africa",
    "North America", "South America", "European Union"
]
df = df[~df["country"].isin(exclude)]

df = df.dropna()

In [5]:
app = dash.Dash(__name__)
app.title = "Global Energy Dashboard"

app.layout = html.Div(style={"padding": "20px"}, children=[

    html.H1("Global Energy Consumption & Renewable Energy Dashboard"),

    html.P(
        "An interactive dashboard exploring global energy consumption trends "
        "and the transition toward renewable energy sources."
    ),

    html.Hr(),

    html.Label("Select Country"),
    dcc.Dropdown(
        options=[{"label": c, "value": c} for c in sorted(df.country.unique())],
        value="Pakistan",
        id="country_dropdown",
        clearable=False
    ),

    html.Br(),

    html.Label("Select Year Range"),
    dcc.RangeSlider(
        min=df.year.min(),
        max=df.year.max(),
        value=[2000, df.year.max()],
        marks={int(y): str(y) for y in range(1990, 2025, 5)},
        id="year_slider"
    ),

    html.Br(), html.Br(),

    dcc.Graph(id="energy_trend"),
    dcc.Graph(id="energy_share"),
    dcc.Graph(id="renewable_breakdown"),
    dcc.Graph(id="renewable_bar")

])

In [ ]:
@app.callback(
    Output("energy_trend", "figure"),
    Output("energy_share", "figure"),
    Output("renewable_breakdown", "figure"),
    Output("renewable_bar", "figure"),
    Input("country_dropdown", "value"),
    Input("year_slider", "value")
)
def update_dashboard(country, year_range):

    dff = df[
        (df["country"] == country) &
        (df["year"].between(year_range[0], year_range[1]))
    ]

    fig1 = px.line(
        dff,
        x="year",
        y="primary_energy_consumption",
        title="Primary Energy Consumption Over Time",
        labels={"primary_energy_consumption": "Energy Consumption (TWh)"}
    )

    fig2 = px.line(
        dff,
        x="year",
        y=["renewables_share_energy", "fossil_share_energy"],
        title="Renewable vs Fossil Energy Share",
        labels={"value": "Energy Share (%)", "variable": "Energy Type"}
    )

    fig3 = px.line(
        dff,
        x="year",
        y=[
            "solar_share_energy",
            "wind_share_energy",
            "hydro_share_energy"
        ],
        title="Renewable Energy Breakdown Over Time",
        labels={"value": "Energy Share (%)", "variable": "Renewable Source"}
    )

    latest_year = df["year"].max()
    latest_df = df[df["year"] == latest_year] \
        .sort_values("renewables_share_energy", ascending=False) \
        .head(15)

    fig4 = px.bar(
        latest_df,
        x="country",
        y="renewables_share_energy",
        title=f"Top 15 Countries by Renewable Energy Share ({latest_year})",
        labels={"renewables_share_energy": "Renewable Energy Share (%)"}
    )

    return fig1, fig2, fig3, fig4


if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
